In [1]:
import nglutils.nglutils as ngu
import nglview as nv
import numpy as np
from polychrom.hdf5_format import load_hdf5_file
import matplotlib.pyplot as plt # only for colormaps

_ColormakerRegistry()

In [2]:
# Load some nice example data. Conveniently, this file contains the conformation, chains and compartment info
data = load_hdf5_file('example_data.h5')

### Quick and easy

In [3]:
view_simple = ngu.xyz2nglview(data['pos'])
view_simple

NGLWidget()

### More detailed visualizations: chromosomes, compartments, other stuff

NGL has a powerful selection language, thus if we properly tell it what our chains (i.e. chromosomes) are, we can later reference them (e.g. ':A' for chr1) and assign colors. Also, we can assign three-letter atom names, which we can use to encode for example compartments. Below I just call all atoms in the first compartment A, such that I can reference them as '.A'. Similarly, the B compartment is '.B' and the active locus that is present in this example is '.C'.

The details of the selection language are [here](http://nglviewer.org/ngl/api/manual/selection-language.html). In general, that [manual](http://nglviewer.org/ngl/api/manual/index.html) proves useful. To avoid confusion though, keep in mind that this is the manual for the actual NGLViewer, which is written in JavaScript.

Parameters for [representations](http://nglviewer.org/ngl/api/manual/molecular-representations.html) are defined and somewhat explained on the [github](https://github.com/arose/ngl/blob/2d84a993/src/representation/representation.js#L279), but what I found useful to play around with is `view._ngl_repr_dict`, which shows all the currently active representations, with all their parameters.

For reference on nglview, see the [README](https://github.com/arose/nglview) on their github.

In [4]:
# Load the data, giving the actual 4 chromosomes that we have and the compartment indices.
# The compartment indices are stored in the "atom names", where they will later be accessible to the selection machinery
comps = [chr(65+i) for i in data['c5_compartmentIDs']]
top = ngu.mdtop_for_polymer(len(data['pos']), chains=data['chains'], atom_names=comps)
view = ngu.xyz2nglview(-data['pos'], top=top)
view

NGLWidget()

In [5]:
# Now we can customize!
# Show chromosomes
view.clear()
ngu.add_colormap_rep(view, plt.cm.viridis, ':B', colorMode='rgb')
ngu.add_colormap_rep(view, plt.cm.spring, ':D', colorMode='rgb')

In [6]:
# Usually monochrome is most useful, that's why there is a small helper for that
view.clear_representations()
ngu.add_uniform_rep(view, 0xffaa55, ':A')
ngu.add_uniform_rep(view, 0xaaff55, ':B')
ngu.add_uniform_rep(view, 0xaa55ff, ':C')
ngu.add_uniform_rep(view, 0x55aaff, ':D')

In [7]:
# Show compartments
view.center()
view.clear_representations()
view.add_representation('licorice', selection='.A',
                                    colorScheme='uniform',
                                    colorValue=0xffcccc)
view.add_representation('hyperball', selection='.B',
                                     colorScheme='uniform',
                                     colorValue=0xccccff,
                                     radius=1)
view.add_representation('ball+stick', selection='.C',
                                      colorScheme='uniform',
                                      colorValue=0xffff00,
                                      radius=0.3,
                                      aspectRatio=2.0)
view.add_representation('surface', selection='.C',
                                   colorScheme='uniform',
                                   colorValue=0xffdd22,
                                   radius=0.3,
                                   opacity=0.3)

In [8]:
# We can add geometric shapes as well!
view.shape.add_sphere([0, 0, 0], [0, 0.5, 0], 35)
view.update_representation(component=1, repr_index=0, opacity=0.2)

### Animations

By simply handing nglview a three dimensional array, we can do animations.

In [9]:
wiggle = np.array([[[s/10, np.sin((s+t)*np.pi/10), t/10*(s/100)**2] for s in range(100)] for t in range(100)])
view_anim = ngu.xyz2nglview(wiggle)
view_anim

NGLWidget(max_frame=99)